<a href="https://colab.research.google.com/github/mifumo081a/HobbyColabProjects/blob/main/ChatGPT_Conversation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import openai


openai.api_key = "API keyを入力"#@param {type:"string"}

In [3]:
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode


RECORD = """
  const sleep = time => new Promise(resolve => setTimeout(resolve, time))
  const b2text = blob => new Promise(resolve => {
    const reader = new FileReader()
    reader.onloadend = e => resolve(e.srcElement.result)
    reader.readAsDataURL(blob)
  })
  var record = time => new Promise(async resolve => {
    stream = await navigator.mediaDevices.getUserMedia({ audio: true })
    recorder = new MediaRecorder(stream)
    chunks = []
    recorder.ondataavailable = e => chunks.push(e.data)
    recorder.start()
    await sleep(time)
    recorder.onstop = async ()=>{
      blob = new Blob(chunks)
      text = await b2text(blob)
      resolve(text)
    }
    recorder.stop()
  })
"""

def speech_to_text(model='whisper-1', language='ja', second=5):
  filename='tmp.wav'
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (second * 1000))
  b = b64decode(s.split(',')[1])

  with open(filename, 'wb+') as fw:
    fw.write(b)

  with open(filename, "rb") as fr:
    transcription = openai.Audio.transcribe(
        model=model, 
        file=fr, 
        language=language
    )
    return transcription['text']

In [4]:
!pip install gtts
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from gtts import gTTS #Import Google Text to Speech
from IPython.display import Audio, display
from pydub import AudioSegment
import time

In [6]:
# 初期のプロンプト
messages = [
  {"role": "system", "content": "あなたは雑談をしてくれるよき隣人です。時たまジョークを交えて返事をしてください。"},
  {"role": "user", "content": "こんにちは。"},
]

while True:
  completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo", 
    messages=messages
  )

  bot_speech = completion.choices[0].message.content
  print(f"assistant: {bot_speech}\n")
  tts = gTTS(bot_speech, lang="ja")
  tts.save("audio.mp3")
  sound = AudioSegment.from_file("audio.mp3", "mp3")
  seconds = sound.duration_seconds
  wn = Audio("audio.mp3", autoplay=True)
  display(wn)
  time.sleep(seconds+3.0)

  while True:
    print("声を出してください。")
    user_speech = speech_to_text()
    if user_speech != '':
      break
  print(f"user: {user_speech}")

  messages.extend([
      {"role": "assistant", "content": bot_speech},
      {"role": "user", "content": user_speech}
  ])
  
  if user_speech == 'おしまい':
    break

assistant: こんにちは！今日も素晴らしい天気ですね！ただ、太陽が熱いだけで、エアコンに頼りたくなりますよね（笑）



声を出してください。


<IPython.core.display.Javascript object>

APIConnectionError: ignored